### test_dataset preprocessing  
1. 년도별 데이터셋 dict 형식으로 변경 dict[기업별][년도별][문장리스트]  
2. 뉴스 19년도부터의 데이터로 인해 잡플래닛 19년도 이하의 데이터들 제외  

In [1]:
import pandas as pd

In [14]:
df_2019 = pd.read_csv('./kosdaq_data/2019_kosdaq.csv', on_bad_lines='skip')
df_2020 = pd.read_csv('./kosdaq_data/2020_kosdaq.csv', on_bad_lines='skip')
df_2021 = pd.read_csv('./kosdaq_data/2021_kosdaq.csv', on_bad_lines='skip')
df_2022 = pd.read_csv('./kosdaq_data/2022_kosdaq.csv', on_bad_lines='skip')
df_2023 = pd.read_csv('./kosdaq_data/2023_kosdaq.csv', on_bad_lines='skip')
df_2024 = pd.read_csv('./kosdaq_data/2024_kosdaq.csv', on_bad_lines='skip')

df_news = pd.concat([df_2019,df_2020,df_2021,df_2022,df_2023,df_2024])

In [15]:
len(df_news)

594200

In [16]:
df_news[:3]

,company,content,date,title,url
0,오상헬스케어,기술력과 제품력을 갖춘 국내 중소벤처 바이오헬스케어 기업들이 세계 최대 가전IT 전...,2019.01.08,"재활훈련·건강검진… 기술中企, 글로벌 시장 노크",https://n.news.naver.com/mnews/article/029/000...
1,오상헬스케어,오상헬스케어036220의 현재 거래량이 최근 5거래일 거래량의 최고치를 넘어섰다&&...,2019.01.09,"[fnRASSI]오상헬스케어, 당일 거래량 증가하며 4거래일 하락세 지속",https://n.news.naver.com/mnews/article/014/000...
2,삼현,증시분석 전문기자 로봇 ET etbotetnews&&com&&전자신문과 금융AI전문...,2019.01.31,"[ET투자뉴스]삼현철강 18년4분기 실적 발표, 당기순이익 4.3억원… 전년 동기 ...",https://n.news.naver.com/mnews/article/030/000...


In [20]:
# df_news = df_news.drop_duplicates(subset='content')
data = []


for index, row in df_news.iterrows():
    sentences = row['content'].split('&&')

    for sentence in sentences:
        if len(sentence) > 40:
            data.append((row['company'], row['date'], row['url'], sentence))
df_news = pd.DataFrame(data, columns=['Company','Date','URL', 'Sentence'])

In [6]:
df_news[:3]

,Company,Date,URL,Sentence
0,오상헬스케어,2019.01.08,https://n.news.naver.com/mnews/article/029/000...,기술력과 제품력을 갖춘 국내 중소벤처 바이오헬스케어 기업들이 세계 최대 가전IT 전...
1,오상헬스케어,2019.01.08,https://n.news.naver.com/mnews/article/029/000...,8일 바이오헬스케어 업계에 따르면 약 20개 국내 중소벤처 바이오헬스케어 기업들이 ...
2,오상헬스케어,2019.01.08,https://n.news.naver.com/mnews/article/029/000...,이번 전시회에서 잠재 고객을 확보하고 글로벌 시장 진출 기회를 잡겠다는 전략이다


In [7]:
df_news = df_news.drop_duplicates(subset='Sentence')

In [8]:
len(df_news)

4571733

In [9]:
df_news['Date'] = df_news['Date'].apply(lambda x:x.split('.')[0])

In [10]:
df_news.head()

,Company,Date,URL,Sentence
0,오상헬스케어,2019,https://n.news.naver.com/mnews/article/029/000...,기술력과 제품력을 갖춘 국내 중소벤처 바이오헬스케어 기업들이 세계 최대 가전IT 전...
1,오상헬스케어,2019,https://n.news.naver.com/mnews/article/029/000...,8일 바이오헬스케어 업계에 따르면 약 20개 국내 중소벤처 바이오헬스케어 기업들이 ...
2,오상헬스케어,2019,https://n.news.naver.com/mnews/article/029/000...,이번 전시회에서 잠재 고객을 확보하고 글로벌 시장 진출 기회를 잡겠다는 전략이다
3,오상헬스케어,2019,https://n.news.naver.com/mnews/article/029/000...,AI를 접목한 홈 재활 서비스와 보조기기를 개발하는 네오펙트는 신제품 네오마노를 공개한다
4,오상헬스케어,2019,https://n.news.naver.com/mnews/article/029/000...,네오마노는 척수 손상 뇌졸중 루게릭 등 중추 신경 장애로 인해 손이 마비된 환자들이...


In [11]:
# '년도' 열을 숫자로 변환하고 변환할 수 없는 값은 NaN으로 설정
df_news['Date'] = pd.to_numeric(df_news['Date'], errors='coerce')

# NaN 값을 제거
df_news.dropna(subset=['Date'], inplace=True)

# '년도' 열을 정수형으로 변환
df_news['Date'] = df_news['Date'].astype(int)

In [12]:
len(df_news)

4571733

In [13]:
df_news.isnull().sum()

Company     0
Date        0
URL         0
Sentence    0
dtype: int64

In [64]:
job_df = pd.read_csv('./data/kosdaq_date_pros_cons_jobplanet_result.csv')

In [65]:
job_df[:3], len(job_df)

(   Unnamed: 0      기업명    날짜  \
 0          28  하나32호스팩  2022   
 1          29  하나32호스팩  2022   
 2          30  하나32호스팩  2022   
 
                                                장점 리뷰  \
 0  대화가 통하는 직원들이 몇명 있다. 일이 힘들지만 마음맞는 사람들과 버틸 수 있는 ...   
 1       음.......... 1. 점심은 줍니다. 2. 월급을 줍니다. 뭐 이정도...   
 2  열심히 일한 사람들한테 그만큼 보상을 해주고 , 업무를 같이 했던 동료들도 다 좋은...   
 
                                                단점 리뷰  
 0                기업 내 정치가 심하다. 대표와 직원들간의 소통이 안되는것같다.  
 1  우선 1,000자 제한이 풀리기를 바라며. 1. 자칭 회장님의 지시만 잘 따르면 됩...  
 2  사실 근무시간은 정확하게 지켜지지 않았으나, 그래도 체계를 잡아가보려고 했던 임직원...  ,
 56259)

In [66]:
# '년도' 열을 숫자로 변환하고 변환할 수 없는 값은 NaN으로 설정
job_df['날짜'] = pd.to_numeric(job_df['날짜'], errors='coerce')

# NaN 값을 제거
job_df.dropna(subset=['날짜'], inplace=True)

# '년도' 열을 정수형으로 변환
job_df['날짜'] = job_df['날짜'].astype(int)

In [67]:
len(job_df)

55484

In [68]:
# 기업별 날짜별 문장을 담을 딕셔너리
company_date_sentence = {}

# 뉴스
for _,token in df_news.iterrows():
    company_name = token['Company']
    sentences = token['Sentence']
    year = token['Date']

    if company_name not in company_date_sentence:
        company_date_sentence[company_name] = {year: [sentences]}
    else:
        if year not in company_date_sentence[company_name]:
            company_date_sentence[company_name][year] = [sentences]
        else:
            company_date_sentence[company_name][year].extend([sentences])

# 잡플래닛
for _, token in job_df.iterrows():
    company_name = token['기업명']
    pos_sentence = token['장점 리뷰']
    neg_sentence = token['단점 리뷰']
    year = token['날짜']
    if company_name not in company_date_sentence:
        company_date_sentence[company_name] = {year: [pos_sentence, neg_sentence]}
    else:
        if year not in company_date_sentence[company_name]:
            company_date_sentence[company_name][year] = [pos_sentence, neg_sentence]
        else:
            company_date_sentence[company_name][year].extend([pos_sentence, neg_sentence])

In [84]:
to_delete = []  # 삭제할 (회사, 연도) 쌍을 저장할 리스트

# 삭제할 항목 식별
for company, years in company_date_sentence.items():
    for year in range(2014, 2019):
        if year in years:
            to_delete.append((company, year))

# 식별된 항목 삭제
for company, year in to_delete:
    del company_date_sentence[company][year]

# 필요한 경우 회사 키도 삭제
# 예를 들어, 모든 연도 정보가 삭제된 후 회사 키가 빈 딕셔너리가 되면 그 키도 삭제할 수 있습니다.
empty_companies = [company for company, years in company_date_sentence.items() if not years]
for company in empty_companies:
    del company_date_sentence[company]

In [85]:
for i in company_date_sentence['삼현']:
    print(i)
    

2019
2020
2021
2022
2023
2024


In [87]:
import json

with open("total_kosdaq_result.json", "w") as f:
    json.dump(company_date_sentence, f)